## Export your assignment notebook to HTML

This cell walks you through the recommended way to export your assignment notebook to pdf for submission.

**Instructions**:    
1. Download the your assignment from collab to your local computer in ipynb format via `File -> Download -> Download .ipynb`.    
2. Run the code cell below. It will prompt you to upload a file. Upload the `.ipynb` file you just downloaded.  
3. It will then use the `nbconvert` package to convert your ipynb notebook to an html file and download it to your local machine. Use this HTML file to prepare your submission.
4. When you print from HTML, use the print dialogue to scale the pages so that long lines are not truncated.

In [6]:
from google.colab import files
import os, shutil, subprocess

# ---------------------------
# 0) Upload notebook
# ---------------------------
uploaded = files.upload()
ipynb_in = list(uploaded.keys())[0]

# Use safe filename (avoid spaces/parentheses)
ipynb_safe = "hw3_submit.ipynb"
shutil.copy(ipynb_in, ipynb_safe)

# ---------------------------
# 1) Ensure nbconvert stack is present
# ---------------------------
subprocess.run(["pip", "-q", "install", "-U", "nbconvert", "nbformat"], check=True)

import nbformat
from nbformat.validator import validate, ValidationError

# ---------------------------
# 2) Load + sanitize outputs (KEEP STATIC FIGURES)
# ---------------------------
nb = nbformat.read(ipynb_safe, as_version=4)

# Mime types to KEEP (static + text)
KEEP_MIME = {
    "text/plain",
    "image/png",
    "image/jpeg",
    "image/svg+xml",
}

# Mime types to ALWAYS DROP (interactive / HTML / JS / widgets)
DROP_MIME_PREFIXES = (
    "application/javascript",
    "text/html",
    "application/vnd.jupyter.widget",
    "application/vnd.plotly",
    "application/vnd.bokeh",
)

def is_drop_mime(mime: str) -> bool:
    return any(mime.startswith(pref) for pref in DROP_MIME_PREFIXES)

def sanitize_output(out: dict) -> dict | None:
    """
    Return sanitized output dict, or None if output should be removed.
    """
    otype = out.get("output_type", "")
    # Keep error outputs? Usually safe, but can be huge; keep them as text only.
    if otype == "error":
        # Keep as-is; nbconvert generally handles it.
        return out

    # Stream outputs (stdout/stderr) are fine
    if otype == "stream":
        return out

    # Display/data outputs: keep only safe mime types
    if otype in ("display_data", "execute_result"):
        data = out.get("data", {})
        if not isinstance(data, dict) or len(data) == 0:
            return None

        new_data = {}
        for mime, payload in data.items():
            # Drop known problematic mimes
            if is_drop_mime(mime):
                continue
            # Keep only whitelisted mimes
            if mime in KEEP_MIME:
                new_data[mime] = payload

        # If nothing left, drop this output
        if len(new_data) == 0:
            return None

        out["data"] = new_data

        # Some outputs include metadata per mime; keep only those mimes
        md = out.get("metadata", {})
        if isinstance(md, dict):
            new_md = {}
            # metadata structure can vary; safest is to keep metadata as-is
            # but prune known per-mime keys if present
            for k, v in md.items():
                if k in KEEP_MIME:
                    new_md[k] = v
                elif k == "image/png" or k == "image/jpeg" or k == "image/svg+xml":
                    new_md[k] = v
                else:
                    # keep non-mime metadata keys (like 'needs_background') cautiously
                    # but drop huge html/js specific entries if any
                    if k not in ("text/html", "application/javascript"):
                        new_md[k] = v
            out["metadata"] = new_md

        return out

    # Any other output types: drop (rare, but safer)
    return None

for cell in nb.cells:
    if cell.get("cell_type") == "code":
        outs = cell.get("outputs", [])
        if isinstance(outs, list):
            new_outs = []
            for out in outs:
                if isinstance(out, dict):
                    sanitized = sanitize_output(out)
                    if sanitized is not None:
                        new_outs.append(sanitized)
            cell["outputs"] = new_outs

# Remove widget metadata (common nbconvert breaker)
nb.metadata.pop("widgets", None)
nb.metadata.pop("widget", None)

clean_ipynb = "hw3_submit_clean.ipynb"
nbformat.write(nb, clean_ipynb)

# Optional validation
try:
    validate(nb)
    print("Notebook validation: OK")
except ValidationError as e:
    print("Notebook validation: FAILED (still attempting nbconvert).")
    print(str(e)[:2000])

# ---------------------------
# 3) Convert to HTML
#   NOTE: --output takes basename (no .html)
# ---------------------------
cmd = [
    "jupyter", "nbconvert",
    "--to", "html",
    clean_ipynb,
    "--output", "hw3_submit",
    "--output-dir", "."
]
res = subprocess.run(cmd, capture_output=True, text=True)

print("nbconvert return code:", res.returncode)
if res.stderr:
    print("STDERR (first 4000 chars):\n", res.stderr[:4000])
print("Files now:", [f for f in os.listdir(".") if f.startswith("hw3_submit")])

# ---------------------------
# 4) Download
# ---------------------------
assert os.path.exists("hw3_submit.html"), "hw3_submit.html not generated"
files.download("hw3_submit.html")

Saving Copy_of_hw3.ipynb to Copy_of_hw3 (2).ipynb
Notebook validation: OK
nbconvert return code: 0
STDERR (first 4000 chars):
 [NbConvertApp] Converting notebook hw3_submit_clean.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 995143 bytes to hw3_submit.html

Files now: ['hw3_submit_clean.ipynb', 'hw3_submit.html', 'hw3_submit.ipynb']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>